# Data Scientist (DS)

Outline of what the DS will do

1. DS logs into DO1 and DO2's datasites as guest  
2. DS explores the datasets  
3. DS prepares syft_flwr code  
4. DS bootstraps the syft_flwr project  
5. DS runs flwr and syft_flwr simulations (optional)   
6. DS submits jobs to the DOs' datasites  
7. DS starts the FL server code  
8. DS Observes the Results

## 1. DS logs into DO1 and DO2's datasites as guest

<img src="../images/dsLogsInAsGuests.png" width="53%" alt="DS logs into DOs' datasites as guests">

In [ ]:
import syft_rds as sy

DS = "ds@openmined.org"
DO1 = "do1@openmined.org"
DO2 = "do2@openmined.org"

ds = sy.init_session(host=DS, email=DS, syftbox_dir="./local_syftbox_network")

In [ ]:
do1_client = sy.init_session(
    host=DO1,
    email=DS,
    syftbox_dir="./local_syftbox_network",
    start_syft_event_server=False,
)
do2_client = sy.init_session(
    host=DO2,
    email=DS,
    syftbox_dir="./local_syftbox_network",
    start_syft_event_server=False,
)

In [ ]:
do1_client.is_admin

In [ ]:
do2_client.is_admin

Set some constants and variables

In [ ]:
import os

SYFTBOX_DATASET_NAME = "pima-indians-diabetes-database"

os.environ["SYFTBOX_CLIENT_CONFIG_PATH"] = str(ds.syftbox_client.config_path)
os.environ["LOGURU_LEVEL"] = "DEBUG"
os.environ["SYFT_FLWR_MSG_TIMEOUT"] = "30"

do_clients = [do1_client, do2_client]
do_emails = [DO1, DO2]

## 2. DS explores the datasets

DS can access the DOs' mock data, but can't access the private data

<img src="../images/dsExploresDOsDatasets.png" width="70%" alt="DS explores datasets">

In [ ]:
SYFTBOX_DATASET_NAME

mock_paths = []
for client in do_clients:
    dataset = client.dataset.get(name=SYFTBOX_DATASET_NAME)
    mock_paths.append(dataset.get_mock_path())
    print(
        f"Client {client.host}'s dataset name: \n{dataset.name}\n. Mock path: \n{dataset.get_mock_path()}"
    )
    print()

Check that the DS can't access the private datasets, even with local test

In [ ]:
from loguru import logger

try:
    dataset.get_private_path()
except Exception as e:
    logger.error(e)

## 3. DS prepares `syft_flwr` code

A `syft_flwr` project requires minimal changes to a Flower project:

1. A `syft_flwr` project has the same structure as a [Flower project](https://flower.ai/docs/framework/tutorial-quickstart-pytorch.html):
```
<your-project-name>
├── <your-project-name>
│   ├── __init__.py
│   ├── client_app.py   # Defines your ClientApp
│   ├── server_app.py   # Defines your ServerApp
│   └── task.py         # Defines your model, training and data loading
├── pyproject.toml      # Project metadata like dependencies and configs
└── README.md
``` 


Concretely, for diabetes prediction, we have the project code under the path `../fl-diabetes-prediction`


<img src="../images/dsPreparesSyftFlwrProject.png" width="30%" alt="DS explores datasets">


where the DS will run the `ServerApp` (defined in `server_app.py`), and the `ClientApp` (in `client_app.py`)

<img src="../images/dsSyftFlwrProjectArch.png" width="70%" alt="Your second image description">


In [ ]:
from pathlib import Path

SYFT_FLWR_PROJECT_PATH = Path("../fl-diabetes-prediction")
assert SYFT_FLWR_PROJECT_PATH.exists()

2. Compared to a Flower project, the DS only needs to change a bit of how the `client_app.py`, which runs on the data owners' datasites, loads the data

<img src="../images/client_fn.png" width="90%" alt="client_fn comparison">

where the `load_syftbox_dataset` (used by `syft_flwr`) and the `load_flwr_data` (used by Flower) differ a bit like below:  

<img src="../images/load_syftbox_flwr_dataset.png" width="90%" alt="load datasets comparison">


## 4. DS bootstraps the `syft_flwr` project

After the DS has prepared the `syft_flwr` project code, they can run the `syft_flwr.bootstrap` command to modify the project's metadata in `pyproject.toml` that specifies the unique name of the FL run, who are the data owners and who is the aggregator
```
[tool.syft_flwr]
app_name = "ds@openmined.org_fl-diabetes-prediction_1753435643"
datasites = [
    "do1@openmined.org",
    "do2@openmined.org",
]
aggregator = "ds@openmined.org"
```
and creates the `main.py` to specifies who runs what based on the new metadata information


<img src="../images/dsBootstrapping.png" width="50%" alt="Your second image description">


In [ ]:
import syft_flwr

try:
    !rm -rf {SYFT_FLWR_PROJECT_PATH / "main.py"}
    logger.info(f"syft_flwr version: {syft_flwr.__version__}")
    syft_flwr.bootstrap(SYFT_FLWR_PROJECT_PATH, aggregator=DS, datasites=do_emails)
    print("Bootstrapped project successfully ✅")
except Exception as e:
    logger.error(e)

## 5. DS runs `flwr` and `syft_flwr` simulations (optional)

In [ ]:
RUN_SIMULATION = True

First we run normal Flower simulation with `flwr run`

In [ ]:
if RUN_SIMULATION:
    !flwr run {SYFT_FLWR_PROJECT_PATH}

In [ ]:
# clean up
!rm -rf {SYFT_FLWR_PROJECT_PATH / "fl_diabetes_prediction" / "__pycache__"}  # clean before submitting
!rm -rf {SYFT_FLWR_PROJECT_PATH / "simulation_logs"}
!rm {SYFT_FLWR_PROJECT_PATH / "uv.lock"}

Now the DS can run `syft_flwr` simulation on DO's mock data by launching 2 threads that run DOs code and a thread that runs the DS code 

<img src="../images/dsRunsSyftFLWRSimulation.png" width="50%" alt="client_fn comparison">

In [ ]:
if RUN_SIMULATION:
    print(f"running syft_flwr simulation with mock paths: {mock_paths}")
    syft_flwr.run(SYFT_FLWR_PROJECT_PATH, mock_paths)

Please look into the dir pointed to by `📝 Log directory` to see the logs of different simulated clients, by default it stays in the `fl_diabetes_prediction/simulation_logs`

<img src="../images/dsSimulationLogs.png" width="24%" alt="DS submits jobs">

## 6. DS submits jobs to the DOs' datasites

The DS submits the code in `fl-diabetes-prediction` to the DO's datasites

<img src="../images/dsSubmitsJobs.png" width="60%" alt="DS submits jobs">

In [ ]:
# clean up before submitting jobs
!rm -rf {SYFT_FLWR_PROJECT_PATH / "fl_diabetes_prediction" / "__pycache__"}
!rm -rf {SYFT_FLWR_PROJECT_PATH / "simulation_logs"}
!rm -rf {SYFT_FLWR_PROJECT_PATH / "uv.lock"}

In [ ]:
logger.info(f"sending job to {do1_client.host}")
job = do1_client.job.submit(
    name="fl-diabetes-prediction",
    user_code_path=SYFT_FLWR_PROJECT_PATH,
    dataset_name=SYFTBOX_DATASET_NAME,
    entrypoint="main.py",
)
logger.success(job)

In [ ]:
logger.info(f"sending job to {do2_client.host}")
job2 = do2_client.job.submit(
    name="fl-diabetes-prediction",
    user_code_path=SYFT_FLWR_PROJECT_PATH,
    dataset_name=SYFTBOX_DATASET_NAME,
    entrypoint="main.py",
)
logger.success(job2)

<img src="../images/dsWaitsForJobsToBeApproved.png" width="35%" alt="DS waits for jobs to be approved">

## 7. DS starts the FL server code

DS submit the job (without `dataset_name` since it's not needed) to herself so she can run it 

In [ ]:
job = ds.job.submit(
    name="fl-diabetes-prediction-ds",
    user_code_path=SYFT_FLWR_PROJECT_PATH,
    entrypoint="main.py",
)
job

In [ ]:
ds.job.get_all()

In [ ]:
ds.job.approve(job)

In [ ]:
ds.run_private(job, blocking=True)

Show the logs of the job

In [ ]:
ds.job.show_logs(job)

In [ ]:
ds.job.get_output_dir(job)

By running the FL server code, the DS aggregates the models trained on DOs' private local data into an improved global model in multiple rounds

<video width="90%" controls>
  <source src="../images/flTraining.mp4" type="video/mp4">
  Your browser does not support the video tag.
</video>

## 8. DS Observes the Results

Now the DS can monitor the aggregated models trained on DO's private datasets in the `weights` folder


<img src="../images/dsAggregatedWeights.png" width="35%" alt="DS weights">

In [ ]:
# clean things up
!rm -rf {str(SYFT_FLWR_PROJECT_PATH / "main.py")} {str(SYFT_FLWR_PROJECT_PATH / "**/__pycache__")}
!rm -rf ./local_syftbox_network/ .syftbox/